## zymomock-preclust

precluster queries like so:
```
~/2022-sourmash-uniqify/uniqify-genomes.py --max-containment --merge --prefix ~/genome-grist/sgc.paper/zymomock-q.
```

In [1]:
name = 'zymomock-preclust'
acc = 'SRR12324253'
ksize=31

In [2]:
metag_filename = f'{name}/{acc}.abundtrim.sig'
queries_filename = f'{name}/{name}-queries.zip'
nbhds_filename = f'{name}/{name}-nbhds.zip'

In [3]:
import sourmash
import csv
import subprocess
import os
import collections

In [4]:
def get_ident(name):
    "pick off identifier, stripping off nbhd: prefix if present."
    name = name.split(' ')[0]
    if name.startswith('nbhd:'):
        name = name[5:]
    return name

def load_gather(filename):
    gather_rows = []
    with open(filename, newline="") as fp:
        r = csv.DictReader(fp)
        gather_rows.extend(r)

    full_idents = {}

    gather_d = {}
    gather_keys = []
    for row in gather_rows:
        ident = get_ident(row['name'])
        full_idents[ident] = " ".join(row['name'].split(' ')[:3])
        gather_d[ident] = row
        gather_keys.append(ident)
        
    return gather_keys, gather_d, full_idents

def gather_and_load(name, metag_filename, query_sigs, nbhd_sigs, ksize=31):
    assert os.path.exists(metag_filename)
    assert os.path.exists(query_sigs)
    assert os.path.exists(nbhd_sigs)
    
    query_out = f"{name}/queries.gather.csv"
    nbhd_out = f"{name}/nbhd.gather.csv"
    stdout1_out = f"{name}/queries.gather.out"
    stdout2_out = f"{name}/nbhd.gather.out"
    
    cmd = f"sourmash gather -k {ksize} {metag_filename} {query_sigs} -o {query_out} --ignore-abundance >& {stdout1_out}"
    print(cmd)
    if not os.path.exists(query_out):
        result = subprocess.run(cmd, shell=True)
    else:
        print(f"** {query_out} already exists; not rerunning sourmash gather on queries")
        
    stdout1 = open(stdout1_out, 'rt').read()


    cmd = f"sourmash gather -k {ksize} {metag_filename} {nbhd_sigs} -o {nbhd_out} --ignore-abundance >& {stdout2_out}"
    print(cmd)
    if not os.path.exists(nbhd_out):
        result = subprocess.run(cmd, shell=True)
    else:
        print(f"** {nbhd_out} already exists; not rerunning sourmash gather on nbhds")
        
    stdout2 = open(stdout2_out, 'rt').read()

       
    gather1_keys, gather1_d, full_idents = load_gather(query_out)
    gather2_keys, gather2_d, _ = load_gather(nbhd_out)
    
    metag = sourmash.load_one_signature(metag_filename, ksize=ksize)
    
    queries = list(sourmash.load_file_as_signatures(query_sigs))
    queries_d = {}
    for ss in queries:
        ident = get_ident(ss.name)
        queries_d[ident] = ss
        
    nbhds = list(sourmash.load_file_as_signatures(nbhd_sigs))
    nbhds_d = {}
    for ss in nbhds:
        ident = get_ident(ss.name)
        nbhds_d[ident] = ss
        
    tup = collections.namedtuple('augmented', 'metag, queries_d, nbhds_d, gather1_keys, gather1_d, gather2_keys, gather2_d, full_idents, stdout1, stdout2')
    t = tup(metag, queries_d, nbhds_d, gather1_keys, gather1_d, gather2_keys, gather2_d, full_idents, stdout1, stdout2)
    
    return t
    


## Running the things

In [5]:
t = gather_and_load(name, metag_filename, queries_filename, nbhds_filename, ksize=ksize)
metag, queries_d, nbhds_d, gather1_keys, gather1_d, gather2_keys, gather2_d, full_idents, stdout1, stdout2 = t

sourmash gather -k 31 zymomock-preclust/SRR12324253.abundtrim.sig zymomock-preclust/zymomock-preclust-queries.zip -o zymomock-preclust/queries.gather.csv --ignore-abundance >& zymomock-preclust/queries.gather.out
sourmash gather -k 31 zymomock-preclust/SRR12324253.abundtrim.sig zymomock-preclust/zymomock-preclust-nbhds.zip -o zymomock-preclust/nbhd.gather.csv --ignore-abundance >& zymomock-preclust/nbhd.gather.out


## gather output

In [6]:
print(stdout1)



overlap     p_query p_match
---------   ------- -------
17.6 Mbp      12.1%   54.5%    AE017341.1 Cryptococcus neoformans va...
13.3 Mbp       9.2%   71.0%    JSPL01000060.1 Escherichia coli strai...
12.4 Mbp       8.5%   41.0%    WMJW01000001.1 Saccharomyces cerevisi...
4.7 Mbp        3.2%   98.2%    VFAF01000002.1 Salmonella enterica st...
4.0 Mbp        2.7%   99.7%    CP039755.1 Bacillus subtilis strain N...
2.8 Mbp        1.9%   99.9%    CP039751.1 Listeria monocytogenes str...
2.7 Mbp        1.9%   99.8%    VFAE01000004.1 Staphylococcus aureus ...
1.8 Mbp        1.2%   99.8%    CP039750.1 Limosilactobacillus fermen...
8.4 Mbp        1.1%    9.1%    CP003820.1 Cryptococcus neoformans va...
2.8 Mbp        0.7%   36.0%    CP039752.1 Enterococcus faecalis stra...
0.5 Mbp        0.4%    9.6%    VEMH01000100.1 Bacillus paranthracis ...

found 11 matches total;
the recovered matches hit 42.9% of the query



== This is sourmash version 4.2.4.dev0+g73aeb155.d20220116. ==

== Please cit

In [7]:
print(stdout2)



overlap     p_query p_match
---------   ------- -------
32.1 Mbp      22.1%  100.0%    nbhd:JSPL01000060.1 Escherichia coli ...
20.1 Mbp      13.8%  100.0%    nbhd:AE017341.1 Cryptococcus neoforma...
13.3 Mbp       9.2%   99.8%    nbhd:WMJW01000001.1 Saccharomyces cer...
12.8 Mbp       8.4%   95.4%    nbhd:VFAF01000002.1 Salmonella enteri...
4.7 Mbp        3.2%   99.4%    nbhd:CP039755.1 Bacillus subtilis str...
3.8 Mbp        2.6%   99.3%    nbhd:VFAE01000004.1 Staphylococcus au...
3.6 Mbp        2.5%   99.4%    nbhd:CP039750.1 Limosilactobacillus f...
3.3 Mbp        2.2%   98.7%    nbhd:CP039751.1 Listeria monocytogene...
5.4 Mbp        0.9%   24.8%    nbhd:CP039752.1 Enterococcus faecalis...
0.7 Mbp        0.4%   91.2%    nbhd:VEMH01000100.1 Bacillus paranthr...
15.1 Mbp       0.3%    3.0%    nbhd:CP003820.1 Cryptococcus neoforma...

found 11 matches total;
the recovered matches hit 65.5% of the query



== This is sourmash version 4.2.4.dev0+g73aeb155.d20220116. ==

== Please cit

## building mapping from query to neighborhood

In [8]:
remaining_hashes = set(metag.minhash.hashes)
unique_hashes_1x = []
for ident in gather1_keys:
    row = gather1_d[ident]
    match = queries_d[ident]
    match_hashes = set(match.minhash.hashes)
    unique_overlap = remaining_hashes & match_hashes
    #print(ident, len(remaining_hashes), len(unique_overlap), row['unique_intersect_bp'], row['remaining_bp'])
    remaining_hashes -= unique_overlap
    unique_hashes_1x.append((ident, unique_overlap))


In [9]:
remaining_hashes = set(metag.minhash.hashes)
unique_hashes_2x = []
for ident in gather2_keys:
    row = gather2_d[ident]
    match = nbhds_d[ident]
    match_hashes = set(match.minhash.hashes)
    unique_overlap = remaining_hashes & match_hashes
    #print(ident, len(remaining_hashes), len(unique_overlap), row['unique_intersect_bp'], row['remaining_bp'])
    remaining_hashes -= unique_overlap
    unique_hashes_2x.append((ident, unique_overlap))


In [10]:
for (nbhd_ident, nbhd_match) in unique_hashes_2x:
    total = 0
    for (query_ident, query_match) in unique_hashes_1x:
        overlap = query_match & nbhd_match
        
        if overlap:
            print(f"{nbhd_ident} <= {query_ident} - {len(overlap)}")
            total += len(overlap)
    #print('xxx', nbhd_ident, len(nbhd_match), total)
    #print('---')

JSPL01000060.1 <= JSPL01000060.1 - 13335
JSPL01000060.1 <= WMJW01000001.1 - 5
JSPL01000060.1 <= VFAF01000002.1 - 66
JSPL01000060.1 <= CP039755.1 - 5
JSPL01000060.1 <= CP039751.1 - 7
JSPL01000060.1 <= CP039752.1 - 384
AE017341.1 <= AE017341.1 - 17440
AE017341.1 <= CP003820.1 - 1304
WMJW01000001.1 <= WMJW01000001.1 - 12097
VFAF01000002.1 <= VFAF01000002.1 - 4546
VFAF01000002.1 <= CP039755.1 - 1
CP039755.1 <= CP039755.1 - 3959
CP039755.1 <= CP039751.1 - 1
VFAE01000004.1 <= CP039751.1 - 5
VFAE01000004.1 <= VFAE01000004.1 - 2712
CP039750.1 <= CP039750.1 - 1800
CP039751.1 <= CP039751.1 - 2800
CP039751.1 <= CP039752.1 - 1
CP039752.1 <= CP039752.1 - 627
VEMH01000100.1 <= VEMH01000100.1 - 524
CP003820.1 <= AE017341.1 - 41
CP003820.1 <= CP003820.1 - 350


In [11]:
import plotly.graph_objects as go

def make_fig(max_num=None):
    #labels = obj.make_labels()
    #src_l, dest_l, cnt_l, color_l, label_l = obj.make_lists()
    labels = []
    src_l = []
    dest_l = []
    cnt_l = []
    color_l = []
    label_l = []
    
    source_idx = {}
    for n, (query_ident, _) in enumerate(unique_hashes_1x):
        source_idx[query_ident] = n
        labels.append(full_idents[query_ident])
    dest_idx = {}
    source_idx["unassigned"] = len(unique_hashes_1x)
    labels.append("unassigned")
    base = len(unique_hashes_1x) + 1
    for n, (nbhd_ident, _) in enumerate(unique_hashes_2x):
        dest_idx[nbhd_ident] = base + n
        labels.append(full_idents[nbhd_ident])
        
    #source_idx["unassigned"] = base + n + 1
    
    # iterate over all sinks, account for all sources
    num = 0
    leftovers = []
    for (nbhd_ident, nbhd_match) in unique_hashes_2x:
        total = 0
        for (query_ident, query_match) in unique_hashes_1x:
            overlap = query_match & nbhd_match
        
            if overlap:
                #print(f"{nbhd_ident} <= {query_ident} - {len(overlap)}")
                total += len(overlap)
                from_idx = source_idx[query_ident]
                to_idx = dest_idx[nbhd_ident]
                
                src_l.append(from_idx)
                dest_l.append(to_idx)
                cnt_l.append(len(overlap))
                if query_ident == nbhd_ident:
                    color_l.append("lightseagreen")
                else:
                    color_l.append("palevioletred")
                label_l.append("")
    
        #print('xxx', nbhd_ident, len(nbhd_match), total)
        leftover = len(nbhd_match) - total
        
        if leftover:
            leftovers.append((nbhd_ident, leftover))

        #print('---')
        num += 1
        if max_num and num >= max_num:
            break
            
    if leftovers:
        for nbhd_ident, leftover in leftovers:
            from_idx = source_idx["unassigned"]
            to_idx = dest_idx[nbhd_ident]

            src_l.append(from_idx)
            dest_l.append(to_idx)
            cnt_l.append(leftover)
            color_l.append("grey")
            label_l.append("")
            #print('XYZ', from_idx, to_idx)
        

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = labels,
          color = "blue"
        ),
        link = dict(
          source = src_l,
          target = dest_l,
          value = cnt_l,
          color = color_l,
          label = label_l,
      ))])
    
    return fig

In [13]:
NUM=11
fig = make_fig(NUM)

fig.update_layout(title_text=f"original genome contributions to top {NUM} neighborhood covers for {name} ({acc})", font_size=10)
fig.update_layout(width=800, height=1000)
fig.show()